## IMPORTAMOS LAS LIBRERIAS

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# from downcast import reduce

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Cargamos los datasets

In [2]:
'''
sales_data_path = ("C:/Users/mitev/Desktop/Simeon/NUCLIO DIGITAL SCHOOL/TFM/datasets_TFM/data_dsmarket/item_sales.csv")
calendar_data_path = ("C:/Users/mitev/Desktop/Simeon/NUCLIO DIGITAL SCHOOL/TFM/datasets_TFM/data_dsmarket/daily_calendar_with_events.csv")
prices_data_path = ("C:/Users/mitev/Desktop/Simeon/NUCLIO DIGITAL SCHOOL/TFM/datasets_TFM/data_dsmarket/item_prices.csv")
'''

'\nsales_data_path = ("C:/Users/mitev/Desktop/Simeon/NUCLIO DIGITAL SCHOOL/TFM/datasets_TFM/data_dsmarket/item_sales.csv")\ncalendar_data_path = ("C:/Users/mitev/Desktop/Simeon/NUCLIO DIGITAL SCHOOL/TFM/datasets_TFM/data_dsmarket/daily_calendar_with_events.csv")\nprices_data_path = ("C:/Users/mitev/Desktop/Simeon/NUCLIO DIGITAL SCHOOL/TFM/datasets_TFM/data_dsmarket/item_prices.csv")\n'

In [3]:
#sales = pd.read_pickle('C:/Users/mitev/Desktop/Simeon/NUCLIO DIGITAL SCHOOL/TFM/datasets_TFM/data_dsmarket/item_sales.csv.pkl')
#prices = pd.read_pickle('C:/Users/mitev/Desktop/Simeon/NUCLIO DIGITAL SCHOOL/TFM/datasets_TFM/data_dsmarket/item_prices.csv.pkl')
#calendar = pd.read_pickle('C:/Users/mitev/Desktop/Simeon/NUCLIO DIGITAL SCHOOL/TFM/datasets_TFM/data_dsmarket/daily_calendar_with_events.csv.pkl')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
sales = pd.read_csv('/content/drive/MyDrive/NUCLIO DIGITAL SCHOOL/TFM/NUCLIO TFM 1023/DSMarket/data_dsmarket/item_sales.csv')
calendar = pd.read_csv('/content/drive/MyDrive/NUCLIO DIGITAL SCHOOL/TFM/NUCLIO TFM 1023/DSMarket/data_dsmarket/daily_calendar_with_events.csv')
prices = pd.read_csv('/content/drive/MyDrive/NUCLIO DIGITAL SCHOOL/TFM/NUCLIO TFM 1023/DSMarket/data_dsmarket/item_prices.csv')

## Funciones

In [6]:
def weekyearnum(dt):
    return dt.strftime("%Y%W")

def myweeyearknum(dt):
    offsetdt = dt + timedelta(days=+2);  
    return weekyearnum(offsetdt)

def weeknum(dt):
    return dt.isocalendar()[1]

def myweeknum(dt):
    offsetdt = dt + timedelta(days=+2);  
    return weeknum(offsetdt)

In [7]:
def reemplazar_eventos(df, columna_fechas, columna_eventos):

    df[columna_fechas] = pd.to_datetime(df[columna_fechas])

    df.loc[(df[columna_fechas].dt.month == 12) & (df[columna_fechas].dt.day == 25) & (df[columna_eventos].isnull()), columna_eventos] = 'Christmas'
    df.loc[(df[columna_fechas].dt.month == 7) & (df[columna_fechas].dt.day == 4) & (df[columna_eventos].isnull()), columna_eventos] = 'Independence Day'
    df.loc[(df[columna_fechas].dt.month == 2) & (df[columna_fechas].dt.day == 14) & (df[columna_eventos].isnull()), columna_eventos] = 'San Valentin'
    df.loc[(df[columna_fechas].dt.month == 3) & (df[columna_fechas].dt.day == 17) & (df[columna_eventos].isnull()), columna_eventos] = 'San Patrick'

    
    dias_laborales = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    es_fin_de_semana = ~df['weekday'].isin(dias_laborales)

    
    df.loc[df[columna_eventos].isnull() & es_fin_de_semana, columna_eventos] = 'Weekend'
    df.loc[df[columna_eventos].isnull(), columna_eventos] = 'Laboral Day'

    return df

In [8]:
def melt_dataframe(df):

    df = pd.melt(frame=df,
                    id_vars=['id', 'item', 'category','department','store','store_code','region'],
                    value_vars=[f'd_{i}' for i in range(1, 1914)],
                    var_name='d',
                    value_name='sales')

    return df

## Dataset Calendario

In [9]:
calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [10]:
calendar.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [11]:
print("The range of dates available are: {} - {}".format(calendar['date'].min(),calendar['date'].max()))

The range of dates available are: 2011-01-29 - 2016-04-24


In [12]:
calendar.isnull().sum()

date              0
weekday           0
weekday_int       0
d                 0
event          1887
dtype: int64

In [13]:
calendar['date'] = pd.to_datetime(calendar['date'], format = "%Y-%m-%d")

calendar['yearweek'] = calendar['date'].apply(lambda x: myweeyearknum(x))

In [14]:
calendar = reemplazar_eventos(calendar,'date','event')

In [15]:
calendar['event'].value_counts()

event
Laboral Day         1336
Weekend              529
SuperBowl              6
San Valentin           6
San Patrick            6
Independence Day       5
Ramadan starts         5
Thanksgiving           5
Christmas              5
NewYear                5
Easter                 5
Name: count, dtype: int64

## Dataset Prices

In [16]:
prices.head()

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,328.00",12.74
1,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,329.00",12.74
2,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,330.00",10.99
3,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,331.00",10.99
4,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,332.00",10.99


In [17]:
porcentaje_nulos = (prices.isnull().sum() / len(prices)) * 100
print(porcentaje_nulos)

item         0.00
category     0.00
store_code   0.00
yearweek     3.50
sell_price   0.00
dtype: float64


In [18]:
prices.dropna(inplace=True)

In [19]:
# Hago la unión entre el dataset de calendario, con el de precio.
calendar['yearweek'] = calendar['yearweek'].astype(float)
calendar_y_prices = pd.merge(prices,calendar[['d','yearweek','date', 'weekday','event']], how="left",
                   left_on = 'yearweek', right_on = 'yearweek')

calendar_y_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46027957 entries, 0 to 46027956
Data columns (total 9 columns):
 #   Column      Dtype         
---  ------      -----         
 0   item        object        
 1   category    object        
 2   store_code  object        
 3   yearweek    float64       
 4   sell_price  float64       
 5   d           object        
 6   date        datetime64[ns]
 7   weekday     object        
 8   event       object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 3.1+ GB


## Dataset Sales

In [20]:
sales.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,

In [21]:
# Creamos una variable temporal en donde solamente dejamos los valores numéricos, para después agrupar en esa variable por la columna de categoria.
# Seleccionamos solo columnas numéricas para la operación de agregación.
numeric_columns = sales.select_dtypes(include=['int64', 'float64']).columns

# Ahora podemos agrupar por la columna de 'category' (o cualquier otra columna no numérica) y calcular la media solo de las columnas numéricas.
grouped_sales = sales.groupby('category')[numeric_columns].mean()

sum_sales = sales.groupby('category')[numeric_columns].sum()

In [22]:
# Primero, crea una lista de las columnas de ventas diarias.
sales_columns = [col for col in sales.columns if col.startswith('d_')]

sales['total_sales'] = sales[sales_columns].sum(axis=1)

sales.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,...,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,total_sales
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0

In [23]:
sales_df = melt_dataframe(sales)

In [24]:
sales_df.head()

,id,item,category,department,store,store_code,region,d,sales
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0


## Join Final

In [25]:
%%time
df_merge_total = pd.merge(sales_df,calendar_y_prices, on=['item', 'category', 'store_code', 'd'], how='inner')

CPU times: user 1min 7s, sys: 9.54 s, total: 1min 16s
Wall time: 1min 16s


In [26]:
df_merge_total.head()

,id,item,category,department,store,store_code,region,d,sales,yearweek,sell_price,date,weekday,event
0,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,12,"201,105.00",0.61,2011-01-29,Saturday,Weekend
1,ACCESORIES_1_009_NYC_1,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2,"201,105.00",2.07,2011-01-29,Saturday,Weekend
2,ACCESORIES_1_010_NYC_1,ACCESORIES_1_010,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,"201,105.00",4.22,2011-01-29,Saturday,Weekend
3,ACCESORIES_1_012_NYC_1,ACCESORIES_1_012,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,"201,105.00",7.95,2011-01-29,Saturday,Weekend
4,ACCESORIES_1_015_NYC_1,ACCESORIES_1_015,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,4,"201,105.00",0.93,2011-01-29,Saturday,Weekend


In [27]:
df_merge_total['revenue'] = df_merge_total['sales'] * df_merge_total['sell_price']

In [28]:
df_merge_total.head()

,id,item,category,department,store,store_code,region,d,sales,yearweek,sell_price,date,weekday,event,revenue
0,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,12,"201,105.00",0.61,2011-01-29,Saturday,Weekend,7.34
1,ACCESORIES_1_009_NYC_1,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2,"201,105.00",2.07,2011-01-29,Saturday,Weekend,4.15
2,ACCESORIES_1_010_NYC_1,ACCESORIES_1_010,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,"201,105.00",4.22,2011-01-29,Saturday,Weekend,0.00
3,ACCESORIES_1_012_NYC_1,ACCESORIES_1_012,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,"201,105.00",7.95,2011-01-29,Saturday,Weekend,0.00
4,ACCESORIES_1_015_NYC_1,ACCESORIES_1_015,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,4,"201,105.00",0.93,2011-01-29,Saturday,Weekend,3.72


In [29]:
'''
df_merge_total_csv = 'df_merge_total.csv'
df_merge_total.to_csv(df_merge_total_csv, index=False)
'''

"\ndf_merge_total_csv = 'df_merge_total.csv'\ndf_merge_total.to_csv(df_merge_total_csv, index=False)\n"